In [1]:
import torch
from PIL import Image
import numpy as np
import glob
import yaml
from utils import *
import time
import shutil
import torch.backends.cudnn as cudnn
from test_dataloader import get_loader_test
import pickle
import argparse
from PLIPmodel import Create_PLIP_Model
from visual_model import *
import math
device = "cuda"


/home/toonies/anaconda3/envs/UTE-AI/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class PLIP_MResNet(nn.Module):
    def __init__(self, image_encoder):
        super().__init__()
        self.image_encoder = image_encoder

    def get_image_embedding(self, img):
        global_image_out, _,_,_,_ = self.image_encoder(img)
        return global_image_out
    def forward(self, image):
        global_image_out,x1,x2,x3,x4 = self.image_encoder(image)
        return global_image_out,x1,x2,x3,x4



In [3]:
from PLIPmodel import Create_PLIP_Model

model = Create_PLIP_Model().to(device)
checkpoint = torch.load("checkpoints/PLIP_RN50.pth.tar",map_location='cpu')
model.image_encoder.load_state_dict(checkpoint["ImgEncoder_state_dict"])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
des_path =  "data/PLIP_Features/"
paths = "data/KeyFramesC00_V00/"
video_paths = sorted(glob.glob(f"{paths}/*/"))
video_paths = ['/'.join(i.split('/')[:-1]) for i in video_paths]

In [7]:
import torchvision.transforms as transforms

transform = transforms.Compose([
            transforms.Resize((256,128), interpolation=3),
            transforms.ToTensor(),
            transforms.Normalize((0.357, 0.323, 0.328),
                                    (0.252, 0.242, 0.239))
])

for vd_path in video_paths:
    print(vd_path)
    re_feats = []
    keyframe_paths = glob.glob(f'{vd_path}/*.jpg')
    keyframe_paths = sorted(keyframe_paths, key=lambda x : x.split('/')[-1].replace('.jpg',''))
    for keyframe_path in keyframe_paths:
        model = model.eval()
        image = Image.open(keyframe_path).convert('RGB')
        image = transform(image).to(device)
        image = image.unsqueeze(0)
        with torch.no_grad():
            image_feats= model.get_image_embeddings(image)
            
        image_feats /= image_feats.norm(dim=-1, keepdim=True)
        image_feats = image_feats.detach().cpu().numpy().astype(np.float16).flatten() 

        re_feats.append(image_feats)

    name_npy = vd_path.split('/')[-1]
    outfile = f'{des_path}/{name_npy}.npy'
    np.save(outfile, re_feats) 

/home/toonies/anaconda3/envs/UTE-AI/lib/python3.7/site-packages/torchvision/transforms/transforms.py:330: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


data/KeyFramesC00_V00/C00_V0000
tensor([[ 3.1117e+00,  5.0835e+00, -3.3313e+00,  5.4726e-01,  4.0894e+00,
         -7.1372e-01,  2.6221e+00, -3.4689e+00, -2.6359e+00,  1.3737e+00,
         -8.0384e+00,  2.7842e+00,  3.1397e+00,  4.9723e+00,  3.6460e+00,
          7.4442e-01,  4.7703e-01,  6.6935e+00, -5.8852e-01,  1.7936e+00,
         -3.3169e+00,  7.3401e+00, -2.7940e+00, -1.2453e+00,  6.1499e+00,
         -4.8895e+00, -1.0034e+00,  4.9664e+00,  4.7597e+00, -4.6299e+00,
          1.8149e+00, -1.1773e+00, -4.3763e-01, -2.6299e+00, -8.9929e+00,
          4.8221e+00,  7.9366e-01, -2.8921e+00,  1.5218e-01, -1.7096e+00,
         -5.5527e+00, -9.4646e-01,  2.8025e+00, -1.7047e-01, -5.4476e-01,
         -2.5194e+00, -8.2504e-01,  2.5771e+00, -2.3418e-01,  2.9661e+00,
          1.9874e+00, -4.3560e+00,  5.8527e-01,  5.4388e+00, -3.5036e+00,
          2.5464e+00, -7.3791e-01, -1.6224e+00, -4.4048e+00,  7.6899e+00,
          1.3843e+00, -3.5034e+00, -6.5068e+00,  2.7943e+00, -6.7612e+00,
      

KeyboardInterrupt: 